<a href="https://colab.research.google.com/github/densivanov/data_analytics_projects/blob/main/uber_analysis/BigQuery_analysis_PySparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Installing libraries

In [27]:
import pandas as pd

!pip install pyspark py4j
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Analysis").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


#Getting data

In [28]:
df = spark.read \
          .option("header",True) \
          .csv("/content/uber_data.csv")

In [29]:
df.show()

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2016-03-01 00:00:00|  2016-03-01 00:07:55|              1|          2.5|-73.97674560546875|40.765151977539055|         1|    

# Creating a view

In [30]:
# Create SQL table
spark.read \
          .option("header",True) \
          .csv("/content/uber_data.csv") \
          .createOrReplaceTempView("df")

In [31]:
spark.sql('''
          SELECT *
          FROM df


''').show()

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RatecodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2016-03-01 00:00:00|  2016-03-01 00:07:55|              1|          2.5|-73.97674560546875|40.765151977539055|         1|    

#Analysis


#### Question №1: Output the average, maximum and minimum distance of the trip

In [32]:
spark.sql('''SELECT ROUND(AVG(trip_distance),2) AS avg_distance,
                    ROUND(MAX(trip_distance),2) AS max_distance,
                    ROUND(MIN(trip_distance),2) AS min_distance
             FROM df
             
             
             
             ''') \
     .show()

+------------+------------+------------+
|avg_distance|max_distance|min_distance|
+------------+------------+------------+
|        3.03|        9.99|         0.0|
+------------+------------+------------+



#### Question №2: How many records are there in the table?

In [34]:
spark.sql('''SELECT COUNT(*) AS records
             FROM df
             
             
             
             ''') \
     .show()

+-------+
|records|
+-------+
| 100000|
+-------+



#### Question №3: Output the average final price and the average tip per trip by the hour

In [44]:
spark.sql('''SELECT EXTRACT(HOUR FROM tpep_pickup_datetime) AS hour,
                    ROUND(AVG(total_amount),2) AS avg_total_amount,
                    ROUND(AVG(tip_amount),2) AS avg_tip_amount
             FROM df
             GROUP BY EXTRACT(HOUR FROM tpep_pickup_datetime)
             ORDER BY EXTRACT(HOUR FROM tpep_pickup_datetime) ASC
             
             
             
             ''') \
     .show()

+----+----------------+--------------+
|hour|avg_total_amount|avg_tip_amount|
+----+----------------+--------------+
|   0|           17.43|          1.91|
|   1|           17.11|          1.75|
|   2|            15.8|          1.49|
|   3|           17.06|          1.43|
|   4|           19.81|          1.88|
|   5|           18.41|          2.09|
|   6|           17.04|          2.01|
|   7|           14.91|          1.82|
|   8|           15.13|          1.86|
|   9|           15.63|          1.89|
|  10|           16.08|          1.91|
|  11|           16.12|          1.84|
|  12|            16.3|          1.86|
|  13|           17.22|          1.94|
|  14|           18.43|          2.09|
+----+----------------+--------------+



#### Question №4: Output the average / max / min final price for the trip by type of payment

In [48]:
spark.sql('''SELECT payment_type,
                    ROUND(AVG(total_amount),2) AS avg_total_amount,
                    ROUND(MAX(total_amount),2) AS max_total_amount,
                    ROUND(MIN(total_amount),2) AS min_total_amount
             FROM df
             GROUP BY payment_type
             ORDER BY payment_type ASC
             
             
             
             ''') \
     .show()

+------------+----------------+----------------+----------------+
|payment_type|avg_total_amount|max_total_amount|min_total_amount|
+------------+----------------+----------------+----------------+
|           1|           18.07|           99.96|             0.0|
|           2|           13.07|           98.88|            -3.8|
|           3|           10.63|             9.8|          -10.14|
|           4|           10.03|             9.8|           -20.8|
+------------+----------------+----------------+----------------+



#### Question №5: Show the total number of users by hour

In [52]:
spark.sql('''SELECT EXTRACT(HOUR FROM tpep_pickup_datetime) AS hour,
                    ROUND(SUM(passenger_count),0) AS sum_passenger_count
             FROM df 
             GROUP BY EXTRACT(HOUR FROM tpep_pickup_datetime)
             ORDER BY EXTRACT(HOUR FROM tpep_pickup_datetime) ASC
             
             ''') \
     .show()

+----+-------------------+
|hour|sum_passenger_count|
+----+-------------------+
|   0|            11475.0|
|   1|             6732.0|
|   2|             4129.0|
|   3|             3135.0|
|   4|             3127.0|
|   5|             5683.0|
|   6|             2861.0|
|   7|            19263.0|
|   8|            23678.0|
|   9|            21768.0|
|  10|            19619.0|
|  11|            19535.0|
|  12|            21565.0|
|  13|            20477.0|
|  14|             9870.0|
+----+-------------------+



#### Question №6: Show metrics on travel time

In [59]:
spark.sql('''SELECT ROUND(AVG(DATEDIFF(minute,tpep_pickup_datetime,tpep_dropoff_datetime)),2) AS avg_time_min,
                    ROUND(MAX(DATEDIFF(minute,tpep_pickup_datetime,tpep_dropoff_datetime)),2) AS max_time_min,
                    ROUND(MIN(DATEDIFF(minute,tpep_pickup_datetime,tpep_dropoff_datetime)),2) AS min_time_min
             FROM df 
             
             ''') \
     .show()

+------------+------------+------------+
|avg_time_min|max_time_min|min_time_min|
+------------+------------+------------+
|       16.42|        1439|           0|
+------------+------------+------------+

